# Extracción de datos de cryptos desde Binance

## Imports

In [1]:
import requests
import datetime
import time 
import os
import pandas as pd
from IPython.display import clear_output, display

: 

## Parámetros

### Parametros por crypto

In [3]:
# Definir los parámetros para cada símbolo y rango de fechas
parametros = {
    "PEPEUSDT": {
        # fecha de creación de PEPE
        "fechainicial": datetime.datetime(2023, 4, 17),
        "fechafinal": datetime.datetime.now()
    },
    "DOGEUSDT": {
        # año de estabilización de DOGE
        "fechainicial": datetime.datetime(2015, 1, 1),
        "fechafinal": datetime.datetime.now()
    },
    "XRPUSDT": {
        # año de estabilización de XRP
        "fechainicial": datetime.datetime(2014, 1, 1),
        "fechafinal": datetime.datetime.now()
    }
}

# Parámetros de la solicitud
symbol = "PEPEUSDT"  
#symbol = "DOGEUSDT"
#symbol = "XRPUSDT" 

fechainicial= parametros[symbol]["fechainicial"]
fechafinal= parametros[symbol]["fechafinal"]

### Parámetros para la API

In [4]:
# URL de la API de Binance
base_url = "https://api.binance.com"
#interval = "1d"  # Intervalo de tiempo (1 día)
interval = "1h"  # Intervalo de tiempo (1 hora)
# Binance API limit
limit = 500  # Límite de la API de Binance
sleep_time = 0.25 #Tiempo de espera entre solicitudes (en segundos), ajustar según sea necesario.


### Otras inicializaciones

In [5]:
nombre_archivo = f"datos_{symbol}.csv"
data_dir = "./data"
# Definir los nombres de las columnas
# La documentación de la API de Binance no proporciona los nombres de las columnas directamente.
# Puedes encontrar la descripción de cada valor en la respuesta aquí:
# https://binance-docs.github.io/apidocs/spot/en/#kline-candlestick-data
column_names = [
    "Open time", 
    "Open", "High", "Low", "Close",
    "Volume", 
    "Close time",
    "Quote asset volume", "Number of trades", "Taker buy base asset volume",
    "Taker buy quote asset volume", "Ignore"
]

## Extracción de los datos

In [6]:
# Function to fetch data in chunks
def fetch_binance_data(symbol, interval, startTime, endTime):
    all_data = []
    numFragmentos = 0
    total_records = 0
    while startTime < endTime:
        clear_output(wait=True)
        try:
            url = f"{base_url}/api/v3/klines?symbol={symbol}&interval={interval}&startTime={startTime}&endTime={endTime}&limit={limit}"
            response = requests.get(url)
            response.raise_for_status() # Lanza una excepción si hay un error HTTP (código de estado != 200)
            data = response.json()
            if data:
                numFragmentos += 1
                records_in_fragment = len(data)
                total_records += records_in_fragment
                display(f"Fragmento {numFragmentos} obtenido: {records_in_fragment} registros. Total: {total_records}")
                all_data.extend(data)
                # Usar el tiempo de apertura para evitar superposiciones
                last_open_time = data[-1][0]
                startTime = last_open_time + 1
                time.sleep(sleep_time) #Añadimos un tiempo de espera para evitar sobrecargar la API
            else:
                display(f"Fragmento vacío.  Posiblemente se ha llegado al final de los datos.Total: {total_records}")
                break
        except requests.exceptions.RequestException as e:
            print(f"Error al obtener datos: {e}")
            break  # Detener si hay un error de red
        except Exception as e: #Capturar cualquier otra excepción inesperada
            print(f"Ocurrió un error inesperado: {e}")
            break

    return all_data

In [7]:
# Convierte la fecha a milisegundos 
startTime = int(fechainicial.timestamp() * 1000)
endTime = int(fechafinal.timestamp() * 1000)
# Fetch data using the function
data = fetch_binance_data(symbol, interval, startTime, endTime)

'Fragmento vacío.  Posiblemente se ha llegado al final de los datos.Total: 15106'

## Crear datframe en pandas

In [9]:
# Convertir los datos a un DataFrame de pandas
df = pd.DataFrame(data, columns=column_names)

# Convertir "Open time" y "Close time" a datetime y darle formato ISO 8601
df['Open time_date'] = pd.to_datetime(df['Open time'], unit='ms').apply(lambda x: x.isoformat())
df['Close time_date'] = pd.to_datetime(df['Close time'], unit='ms').apply(lambda x: x.isoformat())

# Calcular la variación del precio
df['Variation'] = df['Close'].astype(float).diff()

# Definir el orden de las columnas
column_order = [
    "Open time_date", "Close time_date", "Open time", "Close time", "Open", "High", "Low", "Close","Variation",
    "Volume", "Quote asset volume", "Number of trades", "Taker buy base asset volume",
    "Taker buy quote asset volume", "Ignore"
]
# Reordenar las columnas del DataFrame
df = df[column_order]

print(df.head(5).to_markdown(index=False, numalign="left", stralign="left")) 


| Open time_date      | Close time_date            | Open time     | Close time    | Open     | High     | Low      | Close    | Variation   | Volume      | Quote asset volume   | Number of trades   | Taker buy base asset volume   | Taker buy quote asset volume   | Ignore   |
|:--------------------|:---------------------------|:--------------|:--------------|:---------|:---------|:---------|:---------|:------------|:------------|:---------------------|:-------------------|:------------------------------|:-------------------------------|:---------|
| 2023-05-05T18:00:00 | 2023-05-05T18:59:59.999000 | 1683309600000 | 1683313199999 | 1.97e-06 | 4.39e-06 | 1.97e-06 | 3.07e-06 | nan         | 1.82516e+13 | 6.05435e+07          | 182243             | 9.45969e+12                   | 3.14929e+07                    | 0        |
| 2023-05-05T19:00:00 | 2023-05-05T19:59:59.999000 | 1683313200000 | 1683316799999 | 3.07e-06 | 3.47e-06 | 3e-06    | 3.35e-06 | 2.8e-07     | 7.30056e+12 | 2.3552e+07  

In [11]:
print(df.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 15106 entries, 0 to 15105
Data columns (total 15 columns):
 #   Column                        Non-Null Count  Dtype  
---  ------                        --------------  -----  
 0   Open time_date                15106 non-null  object 
 1   Close time_date               15106 non-null  object 
 2   Open time                     15106 non-null  int64  
 3   Close time                    15106 non-null  int64  
 4   Open                          15106 non-null  object 
 5   High                          15106 non-null  object 
 6   Low                           15106 non-null  object 
 7   Close                         15106 non-null  object 
 8   Variation                     15105 non-null  float64
 9   Volume                        15106 non-null  object 
 10  Quote asset volume            15106 non-null  object 
 11  Number of trades              15106 non-null  int64  
 12  Taker buy base asset volume   15106 non-null  object 
 13  T

## Generar archivo de salida

In [10]:
#Crear carpeta data_dir si no existe
if not os.path.exists(data_dir):
    os.makedirs(data_dir)

file_path = os.path.join(data_dir,nombre_archivo )
# Guardar el DataFrame en un archivo CSV
df.to_csv(file_path, index=False)

# Imprimir un mensaje de confirmación
print(f"Los datos se han guardado en el archivo '{file_path}'")

Los datos se han guardado en el archivo './data\datos_PEPEUSDT.csv'
